# 32강 스케쥴러

특정 시간이나 정해진 간격에 따라 어떤 함수가 자동으로 실행되는 처리

> ex) 1마다 한번씩 자동으로 수행되는 기능, 매일 자정에 자동으로 수행되는 기능

apscheduler가 파이썬 공식문서에서 소개되고 있으므로 권장

'''power shell
$ pip install --upgrade apscheduler
'''


## #01. 패키지 참조

In [1]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.base import JobLookupError
import datetime as dt
import time

## 2) 스케쥴에 따라 자동으로 실행될 기능

In [2]:
def myjob(name):
    currentTime = dt.datetime.now()
    timeFormat = currentTime.strftime('%Y/%m/%d %H:%M:%S')
    print(f"[{name}] I'm working.... | {timeFormat}")

myjob('kim')

[kim] I'm working.... | 2023/12/01 12:04:59


## #02. 스케쥴러 등록

### 1) 정해진 간격마다 실행하기

매 3초마다 실행

#### 스케쥴러 객체 생성

In [3]:
sched = BackgroundScheduler()
sched.start()

sched.add_job(myjob, "interval", seconds = 3, args = ['kim'], id='myjob1') # 실행할 함수를 넣고 실행할 함수 안의 파라미터를 args에 리스트로 넘긴다
# 중간에 "interval"을 넣고 seconds = 로 넣어준다

<Job (id=myjob1 name=myjob)>

[kim] I'm working.... | 2023/12/01 12:05:12


In [4]:
sched.remove_job('myjob1')
sched.shutdown()

```python
?print
```

```
Signature: print(*args, sep=' ', end='\n', file=None, flush=False)
Docstring:
Prints the values to a stream, or to sys.stdout by default.

sep
  string inserted between values, default a space.
end
  string appended after the last value, default a newline.
file
  a file-like object (stream); defaults to the current sys.stdout.
flush
  whether to forcibly flush the stream.
Type:      builtin_function_or_method
```

위 코드처럼 ?함수이름 하면 해당 함수에 대한 설명이 나온다

### 스케쥴러 실행

In [5]:
sched = BackgroundScheduler()
sched.start()



### 스케쥴러 작업 등록

 > "interval" 사용 가능 파라미터 seconds, minute, hour, day, month, year, week, day_of_week(요일), start_date, end_date 등

start_date, end_date 는 datetime 객체로 설정해야함

In [6]:
sched.add_job(myjob, "interval", seconds = 3, args = ['kim'], id='myjob1') # 실행할 함수를 넣고 실행할 함수 안의 파라미터를 args에 리스트로 넘긴다

<Job (id=myjob1 name=myjob)>

[kim] I'm working.... | 2023/12/01 12:05:43
[kim] I'm working.... | 2023/12/01 12:05:46
[kim] I'm working.... | 2023/12/01 12:05:49


### 스케쥴러에서 작업 제거

In [19]:
try:    
    sched.remove_job('myjob1')
    sched.shutdown() # 스케쥴러 종료
except JobLookupError as je:
    print('스케쥴러 중지에 실패했습니다.', je)

스케쥴러 중지에 실패했습니다. 'No job by the id of myjob1 was found'


### 2) cron 표현식으로 설정하기

#### cron 표현식

Linux, Mac 등에서 작업 스케쥴러를 등록할 때 사용하는 시간 단위 설정 표현식

공백으로 구분하는 7자리의 값으로 구성됨

```shell
* * * * * * * # 매초 매분 매시 매일 매월 매요일 매년 마다
```

각 자리는 순서대로 '<초> <분> <시> <일> <월> <요일> <년>' 을 의미함

#### (각셀의) 값의 설정 방법

| 필드 | 허용되는 값 | 허용되는 특수문자 |
|---|---|---|
| 초 (Seconds) | 0 ~ 59 | `,` `-` `*` `/` |
| 분 (Minutes) | 0 ~ 59 | `,` `-` `*` `/` |
| 시 (Hours) | 0 ~ 23 | `,` `-` `*` `/` |
| 일 (Day of month) | 1 ~ #! | `,` `-` `*` `/` `L` `W` |
| 월 (Month) | 1 ~ 12 또는 JAN ~ DEC | `,` `-` `*` `/` |
| 요일 (Day of week) | 0 ~ 6 또는 SUN ~ SAT | `,` `-` `*` `/` `L` `#` |
| 년 (Year) | 1970 ~ 2099 | `,` `-` `*` `/` |

#### 특수문자의 의미

- `*` : 모든 값을 뜻합니다.
- 
- `?` : 특정한 값이 없음을 뜻합니다.
- 
- `-` : 범위를 뜻합니다. (예) 월요일에서 수요일 까지는 MON_WED로 표현

- `,` : 특별한 값일때만 동작 (예) 월,수,금 MON,WED,FRI

- `/` : 시작시간/단위 (예) 10분마다 매 5분 10/5

- `L` : 일에서 사용하면 마지막 일, 요일에서는 마지막 요일(토)

- `W` : 가장 가까운 평일 (예) 15W 는 15일에서 가장 가까운 평일(월~금)을 찾ㅇㅁ

- `#` : 몇째주의 무슨 요일을 표현 (예) 3#2 : 2번째주 수요일


(http://crontab.cronhub.io/) << 들어가서 입력하면 동작시간을 표현해줌



In [8]:
sched = BackgroundScheduler()
sched.start()

In [9]:
# 2초마다가 아닌, 매 분 2초가 됨
# -> 2 * * * * * *
sched.add_job(myjob, 'cron', second=2 , args = ['kim'], id = 'myjob2')

# 매초(*)마다 2초 간격(/2)으로 ~
sched.add_job(myjob, 'cron', second='*/2' , args = ['hong'], id = 'myjob3')



<Job (id=myjob3 name=myjob)>

[hong] I'm working.... | 2023/12/01 12:06:04
[hong] I'm working.... | 2023/12/01 12:06:06
[hong] I'm working.... | 2023/12/01 12:06:08
[hong] I'm working.... | 2023/12/01 12:06:10
[hong] I'm working.... | 2023/12/01 12:06:12


In [10]:
sched.remove_job('myjob2')
sched.remove_job('myjob3')
sched.shutdown()

### 3) 특정한 시각에 수행(예약수행)

In [12]:
sched = BackgroundScheduler()
sched.start()

targetDate = dt.datetime(2023,12,1,12,7,0)
sched.add_job(myjob, 'date', run_date = targetDate, args=['park'])




<Job (id=d9914af4b02a46c188d0e0da18d4ffbe name=myjob)>

In [ ]:
# 예약된 시각에 1회 수행하고 종료하므로 remove_job을 할 필요가 없다

sched.shutdown()

## #03. 메일링 리스트 개선


In [13]:
# 메일발송(처리속도차이확인)
import MyMailer

# 비동기 처리 기능 제공
import concurrent.futures as futures

[park] I'm working.... | 2023/12/01 12:07:00


In [14]:
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = '운영지원팀 <hyk2202@gmail.com>'
subjectTmpl = '{name}님의 {yy}년 {mm}월 급여명세서 입니다.'

with open('mail/content.txt','r',encoding='utf-8') as f:
    contentTmpl = f.read()

In [16]:
def sendmail():
    startTime = dt.datetime.now()

    with open('mail/mail_list.csv','r',encoding='euc-kr') as f:
        csv = f.readlines()
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            for line in csv:
                name,email,file1,file2 = line.strip().split(',')
                toAddr = '{name} <{email}>'.format(name=name,email=email)
                subject = subjectTmpl.format(name =name, yy = year, mm = month)
                content = contentTmpl.format(name = name, yy = year, mm = month, dd= day)

                # MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])
                executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

    endTime = dt.datetime.now()
    workTime = endTime - startTime
    print(f'작업에 소요된 시간은 총 {workTime.seconds}초 입니다.')

In [17]:
sched = BackgroundScheduler()
sched.start()
sched.add_job(sendmail, 'cron', second='*/5', id = 'mymailer')


<Job (id=mymailer name=sendmail)>

In [18]:
sched.remove_job('mymailer')
sched.shutdown()

작업에 소요된 시간은 총 3초 입니다.
